# __Variant Filtering tutorial__
In this Jupyter tutorial you can learn how to query variants using PyOskar, which is one of the most typical use cases. PyOskar implements a rich and comprehensive API to help you query variants using the variant annotation, genotypes or file metrics among others.
> <span style="color:#ff6600">__In order to optimise the performance we decided to integrate PyOskar in PySpark default API. This means that it is possible to manage data simultaneously with functions from both libraries in case we want to select specific fields, visualize the dataframe, filter the output or performing other operations besides PyOskar API.__</span>

NOTE: It is important to know that the functionality showed in this tutorial is aimed to make easier getting some particular fields from de DataFrame, but knowing the DataFrame structure and how to access it with default syntax is highly recommended to make the most of PyOskar API.

First, we need to import the PyOskar and PySpark modules. Second, we need to create an instance of the _Oskar()_ object, from which depends a big part of the functionality. Finally, we must use the _load()_ function pointing to where the parquet file is stored to convert our data into a DataFrame _df_, and we are ready to start playing.

In [1]:
from pyoskar.core import Oskar
from pyoskar.sql import *
from pyoskar.analysis import *
from pyspark.sql.functions import *

oskar = Oskar(spark)
df = oskar.load("./data/platinum_chr22.small.parquet")

## Next instruction is needed for querying using Spark SQL language.
df.createOrReplaceTempView("SQLdf")

You can use PySpark _show()_ method to print the data from _df_. This is how our testing dataframe looks like. As you can see for this tutorial we have selected a small dataset from Illumina Platinum Genomes with 1,000 random variants from chromosome 22, which pertain to a set of 17 samples.

In [3]:
print("Print first 20 variants:")
df.show()

Print first 20 variants:
+---------------+-----+----------+--------+--------+---------+---------+------+----+------+-----+----+--------------------+--------------------+
|             id|names|chromosome|   start|     end|reference|alternate|strand|  sv|length| type|hgvs|             studies|          annotation|
+---------------+-----+----------+--------+--------+---------+---------+------+----+------+-----+----+--------------------+--------------------+
|22:16054454:C:T|   []|        22|16054454|16054454|        C|        T|     +|null|     1|  SNV|  []|[[hgvauser@platin...|[22, 16054454, 16...|
|22:16065809:T:C|   []|        22|16065809|16065809|        T|        C|     +|null|     1|  SNV|  []|[[hgvauser@platin...|[22, 16065809, 16...|
|22:16077310:T:A|   []|        22|16077310|16077310|        T|        A|     +|null|     1|  SNV|  []|[[hgvauser@platin...|[22, 16077310, 16...|
|22:16080499:A:G|   []|        22|16080499|16080499|        A|        G|     +|null|     1|  SNV|  []|[[h

NOTE: With _df.printSchema()_ command you can check the dataset hierarchy and all its fields.

In [3]:
print("Samples:")
print(oskar.metadata.samples(df))

Samples:
{'hgvauser@platinum:illumina_platinum': ['NA12877', 'NA12878', 'NA12879', 'NA12880', 'NA12881', 'NA12882', 'NA12883', 'NA12884', 'NA12885', 'NA12886', 'NA12887', 'NA12888', 'NA12889', 'NA12890', 'NA12891', 'NA12892', 'NA12893']}


In [4]:
print("Total number of variants:")
df.count()

Total number of variants:


1000

## Region
To start this tutorial we will execute a simple filtering based on a restricted zone for those people who are not familiarized with PySpark syntax. In this example we chose the nucleotides between 17.000.000 and 17.500.000 position. There is two main ways of querying our filter:

 - PySpark API

In [4]:
# PySpark
df.select("id", "chromosome", "start", "end").filter(df.chromosome == 22).filter(df.start > 17000000).filter(df.end < 17500000).show(10)

+---------------+----------+--------+--------+
|             id|chromosome|   start|     end|
+---------------+----------+--------+--------+
|22:17001352:C:G|        22|17001352|17001352|
|22:17002352:C:A|        22|17002352|17002352|
|22:17004097:G:A|        22|17004097|17004097|
|22:17011943:G:C|        22|17011943|17011943|
|22:17012760:G:A|        22|17012760|17012760|
|22:17013084:C:T|        22|17013084|17013084|
|22:17013900:A:G|        22|17013900|17013900|
|22:17019574:C:A|        22|17019574|17019574|
|22:17030949:T:C|        22|17030949|17030949|
|22:17034810:T:C|        22|17034810|17034810|
+---------------+----------+--------+--------+
only showing top 10 rows



 - Spark SQL

In [5]:
# Spark SQL
spark.sql("SELECT id, chromosome, start, end FROM SQLdf WHERE chromosome =='22' AND start > 17000000 AND end < 17500000").show(10)

+---------------+----------+--------+--------+
|             id|chromosome|   start|     end|
+---------------+----------+--------+--------+
|22:17001352:C:G|        22|17001352|17001352|
|22:17002352:C:A|        22|17002352|17002352|
|22:17004097:G:A|        22|17004097|17004097|
|22:17011943:G:C|        22|17011943|17011943|
|22:17012760:G:A|        22|17012760|17012760|
|22:17013084:C:T|        22|17013084|17013084|
|22:17013900:A:G|        22|17013900|17013900|
|22:17019574:C:A|        22|17019574|17019574|
|22:17030949:T:C|        22|17030949|17030949|
|22:17034810:T:C|        22|17034810|17034810|
+---------------+----------+--------+--------+
only showing top 10 rows



## Gene
We may want to execute a filtering which ties up the variants attached to a particular gene. NBEAP3 was chosen as the target. Here we start preciating the functionality of PyOskar API: *genes()* will automatically locate and acces the field with that same name. We just need to specify the DataFrame column where we store the gene info: _annotation_. Now it's only left to define the filter.
<br>
<br>
Usage:
```python
genes(annotation[str])
```

In [10]:
# PySpark
df.select("id", genes("annotation").alias("genes")).filter(array_contains("genes", "NBEAP3")).show(truncate = False)

+---------------+----------------------+
|id             |genes                 |
+---------------+----------------------+
|22:16096040:G:A|[NBEAP3]              |
|22:16099957:C:T|[NBEAP3]              |
|22:16100462:A:G|[NBEAP3]              |
|22:16105660:G:A|[NBEAP3]              |
|22:16112391:G:A|[NBEAP3]              |
|22:16114913:A:T|[NBEAP3]              |
|22:16127471:A:-|[LA16c-60H5.7, NBEAP3]|
+---------------+----------------------+



Here is how we should use our genes function with pure Spark SQL:

In [6]:
# Spark SQL
spark.sql("SELECT id,genes(annotation) AS genes FROM SQLdf WHERE array_contains(genes(annotation), 'NBEAP3')").show(truncate = False)

+---------------+----------------------+
|id             |genes                 |
+---------------+----------------------+
|22:16096040:G:A|[NBEAP3]              |
|22:16099957:C:T|[NBEAP3]              |
|22:16100462:A:G|[NBEAP3]              |
|22:16105660:G:A|[NBEAP3]              |
|22:16112391:G:A|[NBEAP3]              |
|22:16114913:A:T|[NBEAP3]              |
|22:16127471:A:-|[LA16c-60H5.7, NBEAP3]|
+---------------+----------------------+



And here we will introduce you a third form of building our query, which comes to be half PySpark - half Spark SQL:

In [7]:
# Spark SQL / PySpark
df.selectExpr("id", "genes(annotation) AS genes").filter("array_contains(genes, 'NBEAP3')").show(truncate = False)

+---------------+----------------------+
|id             |genes                 |
+---------------+----------------------+
|22:16096040:G:A|[NBEAP3]              |
|22:16099957:C:T|[NBEAP3]              |
|22:16100462:A:G|[NBEAP3]              |
|22:16105660:G:A|[NBEAP3]              |
|22:16112391:G:A|[NBEAP3]              |
|22:16114913:A:T|[NBEAP3]              |
|22:16127471:A:-|[LA16c-60H5.7, NBEAP3]|
+---------------+----------------------+



NOTE: Later in this tutorial we will only show the purest PySpark syntax. It is up to the user to chose the syntax that best fits his purpose.

## Genotype
In case we want to get the genotypes corresponding to any particular sample we could use the *genotype()* function. We will need to specify the field where that info is contained (_studies_), as well as the targeted sample.
<br>
<br>
Usage:
```python
genotype(studies[str], sample[str])
```

In [12]:
df.select("id", genotype("studies", "NA12877").alias("NA12877")).show(10)

+---------------+-------+
|             id|NA12877|
+---------------+-------+
|22:16054454:C:T|    ./.|
|22:16065809:T:C|    0/1|
|22:16077310:T:A|    ./.|
|22:16080499:A:G|    0/1|
|22:16084621:T:C|    ./.|
|22:16091610:G:T|    ./.|
|22:16096040:G:A|    ./.|
|22:16099957:C:T|    0/1|
|22:16100462:A:G|    0/1|
|22:16105660:G:A|    0/1|
+---------------+-------+
only showing top 10 rows



We could also use a more generic function like *samples_data_field()*. This method allow us to get any format field we desire.
<br>
<br>
Usage:
```python
samples_data(studies[str], annotation[str])
```

In [13]:
df.select(df.id, sample_data_field("studies", "NA12878", "GT").alias("NA12878")).show(10)

+---------------+-------+
|             id|NA12878|
+---------------+-------+
|22:16054454:C:T|    ./.|
|22:16065809:T:C|    ./.|
|22:16077310:T:A|    0/1|
|22:16080499:A:G|    ./.|
|22:16084621:T:C|    0/1|
|22:16091610:G:T|    ./.|
|22:16096040:G:A|    ./.|
|22:16099957:C:T|    0/1|
|22:16100462:A:G|    0/1|
|22:16105660:G:A|    ./.|
+---------------+-------+
only showing top 10 rows



And also we dispose of *samples_data()*, which will return all the format fields all in once. Unluckily the VCF that originated this testing dataframe only stored genotype data!
<br>
<br>
Usage:
```python
samples_data_field(studies[str], annotation[str], formatField[str])
```

In [9]:
df.select(df.id, sample_data("studies", "NA12879").alias("NA12879")).show(10)

+---------------+-------+
|             id|NA12879|
+---------------+-------+
|22:16054454:C:T|  [./.]|
|22:16065809:T:C|  [./.]|
|22:16077310:T:A|  [0/1]|
|22:16080499:A:G|  [0/1]|
|22:16084621:T:C|  [./.]|
|22:16091610:G:T|  [./.]|
|22:16096040:G:A|  [./.]|
|22:16099957:C:T|  [0/1]|
|22:16100462:A:G|  [0/1]|
|22:16105660:G:A|  [0/1]|
+---------------+-------+
only showing top 10 rows



Multiple filtering can be queried like this:

In [3]:
df.filter(((genotype("studies", "NA12877") == "0/1") | (genotype("studies", "NA12877") == "1/1")) & (genotype("studies", "NA12878") == "./."))\
    .withColumn("NA12877_gt", genotype("studies", "NA12877")).withColumn("NA12878_gt", genotype("studies", "NA12878")).show(10)

+--------------------+-----+----------+--------+--------+---------+---------+------+----+------+-----+----+--------------------+--------------------+----------+----------+
|                  id|names|chromosome|   start|     end|reference|alternate|strand|  sv|length| type|hgvs|             studies|          annotation|NA12877_gt|NA12878_gt|
+--------------------+-----+----------+--------+--------+---------+---------+------+----+------+-----+----+--------------------+--------------------+----------+----------+
|     22:16065809:T:C|   []|        22|16065809|16065809|        T|        C|     +|null|     1|  SNV|  []|[[hgvauser@platin...|[22, 16065809, 16...|       0/1|       ./.|
|     22:16080499:A:G|   []|        22|16080499|16080499|        A|        G|     +|null|     1|  SNV|  []|[[hgvauser@platin...|[22, 16080499, 16...|       0/1|       ./.|
|     22:16105660:G:A|   []|        22|16105660|16105660|        G|        A|     +|null|     1|  SNV|  []|[[hgvauser@platin...|[22, 1610566

NOTE: _withColumn()_ is a PySpark method used to create or rename columns.

## Population Frequency
The *population_frequency()* function acts upon the variant annotation. Apart from specifying the field where we contain that information (_annotation_), we will need to specify the ID of the particular study we would like to check, the target population and a specific cohort.
<br>
<br>
Usage:
```python
population_frequency(annotation[str], study[str], population[str])
```

In [12]:
df.select("id", population_frequency("annotation", "GNOMAD_GENOMES", "ALL").alias("GNOMAD_GENOMES:ALL"))\
    .filter(col("GNOMAD_GENOMES:ALL") != 0).show(10, truncate = False)

+---------------+---------------------+
|id             |GNOMAD_GENOMES:ALL   |
+---------------+---------------------+
|22:16054454:C:T|0.07566695660352707  |
|22:16065809:T:C|0.14594951272010803  |
|22:16077310:T:A|0.2338419109582901   |
|22:16091610:G:T|0.0031298904214054346|
|22:16099957:C:T|0.6782668232917786   |
|22:16105660:G:A|0.12387744337320328  |
|22:16114913:A:T|0.11458797007799149  |
|22:16127471:A:-|0.004762233234941959 |
|22:16134019:G:T|0.03758351877331734  |
|22:16138943:C:G|0.2861359417438507   |
+---------------+---------------------+
only showing top 10 rows



Otherwise, if we are interested in getting all the Population Frequencies available independently of the cohort, we can use this other method named *population_frequency_as_map()* which will return the whole set of PF as a dictionary format. Then we could apply _explode()_ and convert that dictionary into a new dataframe. In this example we have also filtered by variant to get a better looking output:
<br>
Usage:
```python
population_frequency_as_map(annotation[str])
```

In [5]:
pf_df = df.select(df.id, population_frequency_as_map("annotation").alias("population_frequencies")).filter(df.id == "22:16099957:C:T")
pf_df.select("id", explode(pf_df.population_frequencies).alias("study", "population_frequencies_df")).show(truncate = False)

+---------------+---------------------+-------------------------+
|id             |study                |population_frequencies_df|
+---------------+---------------------+-------------------------+
|22:16099957:C:T|GNOMAD_GENOMES:OTH   |0.6896551847457886       |
|22:16099957:C:T|GNOMAD_GENOMES:ALL   |0.6782668232917786       |
|22:16099957:C:T|GNOMAD_GENOMES:AFR   |0.6747621297836304       |
|22:16099957:C:T|GNOMAD_GENOMES:NFE   |0.6699579954147339       |
|22:16099957:C:T|GNOMAD_GENOMES:MALE  |0.682662844657898        |
|22:16099957:C:T|GNOMAD_GENOMES:FIN   |0.6726332306861877       |
|22:16099957:C:T|GNOMAD_GENOMES:EAS   |0.9523809552192688       |
|22:16099957:C:T|GNOMAD_GENOMES:ASJ   |0.5721649527549744       |
|22:16099957:C:T|GNOMAD_GENOMES:FEMALE|0.6727412939071655       |
|22:16099957:C:T|GNOMAD_GENOMES:AMR   |0.6950549483299255       |
+---------------+---------------------+-------------------------+



## Biotype
Some filters may want to access to *biotypes()* field. Here is an example:
<br>
<br>
Usage:
```python
biotypes(annotation[str])
```

In [19]:
df.select(df.id, biotypes("annotation").alias("biotypes")).filter(array_contains("biotypes", "protein_coding")).show(10, truncate = False)

+---------------+-------------------------------------------------------------------------------------+
|id             |biotypes                                                                             |
+---------------+-------------------------------------------------------------------------------------+
|22:16252532:A:G|[processed_pseudogene, protein_coding, nonsense_mediated_decay]                      |
|22:16289841:A:-|[protein_coding, nonsense_mediated_decay]                                            |
|22:17069538:C:A|[protein_coding]                                                                     |
|22:17270265:T:C|[protein_coding]                                                                     |
|22:17271137:T:-|[protein_coding]                                                                     |
|22:17282666:G:A|[protein_coding]                                                                     |
|22:17285049:G:C|[protein_coding]                               

## Consequence type
There are two ways we could acces _consequence type_ field. The first one is the *consequence_types()* function:
<br>
<br>
Usage:
```python
consequence_type(annotation[str])
```

In [15]:
df.select(df.id, genes("annotation").alias("genes"), consequence_types("annotation").alias("consequence_types")).show(20, truncate = 80)

+---------------+---------------------------------------+--------------------------------------------------------------------------------+
|             id|                                  genes|                                                               consequence_types|
+---------------+---------------------------------------+--------------------------------------------------------------------------------+
|22:16054454:C:T|                                     []|                                                            [intergenic_variant]|
|22:16065809:T:C|                          [LA16c-4G1.3]|                            [regulatory_region_variant, downstream_gene_variant]|
|22:16077310:T:A|                          [LA16c-4G1.4]|                               [2KB_upstream_variant, regulatory_region_variant]|
|22:16080499:A:G|             [LA16c-4G1.4, LA16c-4G1.5]|                                [upstream_gene_variant, downstream_gene_variant]|
|22:16084621:T:C|          

The other way is with *consequence_types_by_gene()* method. Unlike the one above, it will return the field that contains the consequence types that **directly match the specified gene!!!**
<br>
<br>
Usage:
```python
consequence_type_as_map(annotation[str], gene[str])
```

In [22]:
df.select(df.id, genes("annotation").alias("genes"), consequence_types_by_gene("annotation", "NBEAP3").alias("consequence_types")).show(20, truncate = False)

+---------------+---------------------------------------+-----------------------------------------------+
|id             |genes                                  |consequence_types                              |
+---------------+---------------------------------------+-----------------------------------------------+
|22:16054454:C:T|[]                                     |[]                                             |
|22:16065809:T:C|[LA16c-4G1.3]                          |[]                                             |
|22:16077310:T:A|[LA16c-4G1.4]                          |[]                                             |
|22:16080499:A:G|[LA16c-4G1.4, LA16c-4G1.5]             |[]                                             |
|22:16084621:T:C|[LA16c-4G1.5]                          |[]                                             |
|22:16091610:G:T|[]                                     |[]                                             |
|22:16096040:G:A|[NBEAP3]                     

We may want to combine that output with a filter like this:

In [3]:
df.select(df.id, consequence_types_by_gene("annotation", "NBEAP3").alias("consequence_types")).filter(array_contains("consequence_types", "intron_variant"))\
    .show(10, truncate = False)

+---------------+-----------------------------------------------+
|id             |consequence_types                              |
+---------------+-----------------------------------------------+
|22:16105660:G:A|[non_coding_transcript_variant, intron_variant]|
|22:16112391:G:A|[non_coding_transcript_variant, intron_variant]|
|22:16114913:A:T|[non_coding_transcript_variant, intron_variant]|
+---------------+-----------------------------------------------+



## Protein score
Human evolution data is stored in the _conservation_, _functional_ and _protein substitution_ fields. We will give a few examples:

 - Conservation scores: GERP, Phylop and PhastCons

Usage:
```python
conservation(annotation[str], source[str])
```

In [20]:
df.select("id", conservation("annotation", "gerp").alias("gerp"), conservation("annotation", "phastCons").alias("phastCons")).filter("gerp > 0.5")\
    .filter("phastCons > 0.2").show(10, truncate = False)

+---------------+------------------+-------------------+
|id             |gerp              |phastCons          |
+---------------+------------------+-------------------+
|22:16127471:A:-|1.3799999952316284|0.2709999978542328 |
|22:18406473:C:T|1.7799999713897705|0.5329999923706055 |
|22:18824362:C:T|1.850000023841858 |0.4050000011920929 |
|22:18910451:A:C|0.9670000076293945|0.8029999732971191 |
|22:19018034:G:A|0.7429999709129333|0.3630000054836273 |
|22:21600633:C:G|1.0099999904632568|0.6600000262260437 |
|22:21606270:G:A|1.5800000429153442|0.2750000059604645 |
|22:21842655:C:T|1.3700000047683716|0.6650000214576721 |
|22:21906285:C:T|0.7689999938011169|0.37299999594688416|
|22:22172155:G:A|0.675000011920929 |0.30799999833106995|
+---------------+------------------+-------------------+
only showing top 10 rows



 - Functional scores: CADD scaled and CADD raw

Usage:
```python
functional(annotation[str], source[str])
```

In [21]:
df.select("id", functional("annotation", "cadd_scaled").alias("cadd_scaled"), functional("annotation", "cadd_raw").alias("cadd_raw")).filter("cadd_raw < 0").show(10)

+---------------+-------------------+--------------------+
|             id|        cadd_scaled|            cadd_raw|
+---------------+-------------------+--------------------+
|22:16054454:C:T| 1.5700000524520874|-0.13000011444091797|
|22:16065809:T:C| 1.4299999475479126|-0.14999961853027344|
|22:16080499:A:G| 0.7900000214576721| -0.2700004577636719|
|22:16084621:T:C|0.44999998807907104|-0.36999988555908203|
|22:16096040:G:A|  0.949999988079071|-0.22999954223632812|
|22:16112391:G:A|0.03999999910593033| -0.7799997329711914|
|22:16114913:A:T| 0.8700000047683716|               -0.25|
|22:16134019:G:T|0.49000000953674316| -0.3599996566772461|
|22:16144239:T:C|               1.25|-0.18000030517578125|
|22:16229483:C:T|0.03999999910593033| -0.8100004196166992|
+---------------+-------------------+--------------------+
only showing top 10 rows



 - Protein substitution scores: PolyPhen and SIFT

Usage:
```python
protein_substitution(annotation[str], score[str])
```

In [16]:
df.select("id", protein_substitution("annotation", "polyphen").alias("polyphen"), protein_substitution("annotation", "sift").alias("sift"))\
    .where("polyphen[0] >= 0").show()

+---------------+--------------+------------+
|             id|      polyphen|        sift|
+---------------+--------------+------------+
|22:25024098:G:A|[0.124, 0.581]| [0.0, 0.12]|
|22:28378810:T:G|[0.005, 0.005]|[0.37, 0.37]|
|22:29885644:C:A|[0.001, 0.001]|  [1.0, 1.0]|
|22:32831711:C:G|[0.027, 0.351]|[0.54, 0.78]|
+---------------+--------------+------------+



## Study
PyOskar also offers different methods to access the clinical annotation data stored in the studies field. This functionality is divided among the next methods:

 - *study()* returns all the study info related to the variant:

 Usage:
```python
study(studies[str], studyId[str])
```

In [10]:
df.select("id", study("studies", "hgvauser@platinum:illumina_platinum").alias("study")).show(10, truncate = 140)

+---------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|             id|                                                                                                                                       study|
+---------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|22:16054454:C:T|[hgvauser@platinum:illumina_platinum, [[platinum-genomes-vcf-NA12890_S1.genome.vcf.gz,, [AC -> 1, culprit -> QD, MQ0 -> 15, SB -> -65.03,...|
|22:16065809:T:C|[hgvauser@platinum:illumina_platinum, [[platinum-genomes-vcf-NA12877_S1.genome.vcf.gz,, [AC -> 1, culprit -> MQ, MQ0 -> 50, SB -> -7.69, ...|
|22:16077310:T:A|[hgvauser@platinum:illumina_platinum, [[platinum-genomes-vcf-NA12878_S1.genome.vcf.gz,, [AC -> 1, culprit -> QD, MQ0 -> 5, SB -> -145.92,...|
|22:16080499:A:G|[hgvauser@platinum:illumina_p

Understanding the field structure will allow us to get what we need:

In [5]:
df.select(study("studies", "hgvauser@platinum:illumina_platinum")).dtypes

[('UDF(studies, hgvauser@platinum:illumina_platinum)',
  'struct<studyId:string,files:array<struct<fileId:string,call:string,attributes:map<string,string>>>,secondaryAlternates:array<struct<chromosome:string,start:int,end:int,reference:string,alternate:string,type:string>>,format:array<string>,samplesData:array<array<string>>,stats:map<string,struct<alleleCount:int,refAlleleCount:int,altAlleleCount:int,refAlleleFreq:float,altAlleleFreq:float,genotypeCount:map<string,int>,genotypeFreq:map<string,float>,missingAlleleCount:int,missingGenotypeCount:int,maf:float,mgf:float,mafAllele:string,mgfGenotype:string>>,scores:array<struct<id:string,cohortId:string,secondCohortId:string,score:float,pValue:float>>>')]

 - *file()* returns the study data involving a specific record. Every variant contains study info, but not for every record. We could want to filter the output to avoid null results:

 Usage:
```python
file(studies[str], fileId[str])
```

In [3]:
df.select("id", file("studies", "platinum-genomes-vcf-NA12877_S1.genome.vcf.gz").alias("NA12877_study")).show(5, truncate = 140)

+---------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|             id|                                                                                                                               NA12877_study|
+---------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|22:16054454:C:T|                                                                                                                                        null|
|22:16065809:T:C|[platinum-genomes-vcf-NA12877_S1.genome.vcf.gz,, [AC -> 1, culprit -> MQ, MQ0 -> 50, SB -> -7.69, ReadPosRankSum -> 0.731, AN -> 2, set -...|
|22:16077310:T:A|                                                                                                                                        null|
|22:16080499:A:G|[platinum-genomes-vcf-NA12877

In [4]:
df.select("id", file("studies", "platinum-genomes-vcf-NA12877_S1.genome.vcf.gz").alias("NA12877_study")).filter(col("NA12877_study")["fileId"] != "null").show(5, truncate = 140)

+---------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|             id|                                                                                                                               NA12877_study|
+---------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|22:16065809:T:C|[platinum-genomes-vcf-NA12877_S1.genome.vcf.gz,, [AC -> 1, culprit -> MQ, MQ0 -> 50, SB -> -7.69, ReadPosRankSum -> 0.731, AN -> 2, set -...|
|22:16080499:A:G|[platinum-genomes-vcf-NA12877_S1.genome.vcf.gz,, [AC -> 1, culprit -> QD, MQ0 -> 3, SB -> -0.01, ReadPosRankSum -> 1.136, AN -> 2, set ->...|
|22:16099957:C:T|[platinum-genomes-vcf-NA12877_S1.genome.vcf.gz,, [AC -> 1, culprit -> MQ, MQ0 -> 6, SB -> -0.01, ReadPosRankSum -> 0.439, AN -> 2, set ->...|
|22:16100462:A:G|[platinum-genomes-vcf-NA12877

This is how the structure of the file looks like:

In [8]:
df.select(file("studies", "platinum-genomes-vcf-NA12877_S1.genome.vcf.gz").alias("NA12877_study")).dtypes

[('NA12877_study',
  'struct<fileId:string,call:string,attributes:map<string,string>>')]

And these are the keys found in any file attributes. We used PySpark _explode_ method, which returns a new row for each element in the given array or map:

In [38]:
df.select(explode(file("studies", "platinum-genomes-vcf-NA12877_S1.genome.vcf.gz")["attributes"])).select("key").distinct().show(25)

+--------------+
|           key|
+--------------+
|            SB|
|           set|
|     MQRankSum|
|  BaseQRankSum|
|            FS|
|           MQ0|
|          HRun|
|          QUAL|
|        VQSLOD|
|            MQ|
|       culprit|
|          Dels|
|            QD|
|HaplotypeScore|
|            DS|
|            AC|
|        FILTER|
|            AN|
|ReadPosRankSum|
|            AF|
|            DP|
+--------------+



 - *file_attributes()* returns a specific value contained inside the _attributes_ Map:

 Usage:
```python
file_attribute(studies[str], fileId[str], info[str])
```

In [17]:
df.select("id", file_attribute("studies", "platinum-genomes-vcf-NA12877_S1.genome.vcf.gz", "DP").alias("depth")).filter(col("depth") != "null").show(10)
# ALTERNATIVE SQL SYNTAX FOR THE FILTER:
# df.select("id", file_attribute("studies", "platinum-genomes-vcf-NA12877_S1.genome.vcf.gz", "DP").alias("Depth")).where("Depth is not null").show(10)

+---------------+-----+
|             id|depth|
+---------------+-----+
|22:16065809:T:C|   91|
|22:16080499:A:G|   68|
|22:16099957:C:T|   52|
|22:16100462:A:G|  112|
|22:16105660:G:A|  159|
|22:16138943:C:G|  139|
|22:16144239:T:C|   38|
|22:16149692:G:T|   46|
|22:16195955:G:A|   75|
|22:16196041:C:T|  109|
+---------------+-----+
only showing top 10 rows

